In [ ]:
!pip install "fastapi[standard]"
!pip install fastapi nest_asyncio pyngrok uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.6/950.6 kB 35.8 MB/s eta 0:00:00


In [ ]:
%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps git+https://github.com/huggingface/transformers.git # Only for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N
!pip install huggingface-hub==0.34.0
!pip install --upgrade transformers
!pip install bitsandbytes

In [ ]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [ ]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("TechBuz/gemma-3N-finetune", torch_dtype="auto"),

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/AI4Good/gemmacomp/saved_gemma_model"
tokenizer = AutoTokenizer.from_pretrained("TechBuz/gemma-3N-finetune")
model = AutoModelForCausalLM.from_pretrained("TechBuz/gemma-3N-finetune", device_map="cpu")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Loading adapter weights from TechBuz/gemma-3N-finetune led to missing keys in the model: model.language_model.layers.0.self_attn.q_proj.lora_A.default.weight, model.language_model.layers.0.self_attn.q_proj.lora_B.default.weight, model.language_model.layers.0.self_attn.k_proj.lora_A.default.weight, model.language_model.layers.0.self_attn.k_proj.lora_B.default.weight, model.language_model.layers.0.self_attn.v_proj.lora_A.default.weight, model.language_model.layers.0.self_attn.v_proj.lora_B.default.weight, model.language_model.layers.0.self_attn.o_proj.lora_A.default.weight, model.language_model.layers.0.self_attn.o_proj.lora_B.default.weight, model.language_model.layers.0.mlp.gate_proj.lora_A.default.weight, model.language_model.layers.0.mlp.gate_proj.lora_B.default.weight, model.language_model.layers.0.mlp.up_proj.lora_A.default.weight, model.language_model.layers.0.mlp.up_proj.lora_B.default.weight, model.language_model.layers.0.mlp.down_proj.lora_A.default.weight, model.language_model

In [ ]:
from pyngrok import ngrok
import os
from google.colab import userdata

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
# You can store your authtoken securely in Colab's Secrets Manager and access it using userdata.get('NGROK_AUTH_TOKEN')
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
app = FastAPI()

# Input structure
class PromptInput(BaseModel):
    age: int
    SystolicBP: int
    DiastolicBP: int
    BS: float
    BodyTemp: float
    heart_rate: int
    max_new_tokens: int = 20

@app.post("/generate")
def generate_text(data: PromptInput):
    prompt = f"""
Age: {data.age}
SystolicBP: {data.SystolicBP}
DiastolicBP: {data.DiastolicBP}
Blood Sugar: {data.BS}
Body Temperature: {data.BodyTemp}
Heart Rate: {data.heart_rate}

Please respond with only one of the following:
"Low Risk", "Mid Risk", or "High Risk".
Return only the risk level. No explanation.
"""

    inputs = tokenizer(prompt.strip(), return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=data.max_new_tokens,
        do_sample=True,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean the response
    cleaned = response.strip().lower()
    if "high" in cleaned:
        return "High Risk"
    elif "mid" in cleaned:
        return "Mid Risk"
    elif "low" in cleaned:
        return "Low Risk"
    else:
        return "Unknown Risk"


In [ ]:
# launch_ngrok_server.py
import nest_asyncio
from pyngrok import ngrok
from uvicorn import Config, Server
import asyncio
#from main import app  # This will import the FastAPI app
from fastapi import FastAPI, Request
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

nest_asyncio.apply()

# Define the FastAPI app and input structure here
app = FastAPI()

# Input structure
class PromptInput(BaseModel):
    age: int
    SystolicBP: int
    DiastolicBP: int
    BS: float
    BodyTemp: float
    heart_rate: int
    max_new_tokens: int = 20

@app.post("/generate")
def generate_text(data: PromptInput):
    prompt = f"""
Age: {data.age}
SystolicBP: {data.SystolicBP}
DiastolicBP: {data.DiastolicBP}
Blood Sugar: {data.BS}
Body Temperature: {data.BodyTemp}
Heart Rate: {data.heart_rate}

Please respond with only one of the following:
"Low Risk", "Mid Risk", or "High Risk".
Return only the risk level. No explanation.
"""

    inputs = tokenizer(prompt.strip(), return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=data.max_new_tokens,
        do_sample=True,
        temperature=1,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean the response
    cleaned = response.strip().lower()
    if "high" in cleaned:
        return "High Risk"
    elif "mid" in cleaned:
        return "Mid Risk"
    elif "low" in cleaned:
        return "Low Risk"
    else:
        return "Unknown Risk"


# Expose port 8001
public_url = ngrok.connect(8001)
print(f"🚀 Open your API docs here: {public_url}/docs")

# Run server
config = Config(app=app, host="0.0.0.0", port=8001, log_level="info")
server = Server(config=config)
asyncio.run(server.serve())

🚀 Open your API docs here: NgrokTunnel: "https://4a819859e31c.ngrok-free.app" -> "http://localhost:8001"/docs


INFO:     Started server process [7172]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     2604:3d09:a173:aa00:c137:9929:aa47:c3e3:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2604:3d09:a173:aa00:c137:9929:aa47:c3e3:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2604:3d09:a173:aa00:c137:9929:aa47:c3e3:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2604:3d09:a173:aa00:c137:9929:aa47:c3e3:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for background tasks to complete. (CTRL+C to force quit)
INFO:     Finished server process [7172]


KeyboardInterrupt: 

In [ ]:
#from unsloth.chat_templates import get_chat_template

# Prepare tokenizer once (outside the function)
#tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")

def predict_pregnancy_risk(row, model, tokenizer, max_new_tokens=10):
    """
    Generates a pregnancy risk prediction using the given model and vitals in a row.

    Parameters:
        row (pd.Series): A row from your DataFrame containing vitals.
        model (transformers.PreTrainedModel): The loaded Gemma model.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer with chat template.
        max_new_tokens (int): Max number of tokens to generate.

    Returns:
        str: Decoded model output (e.g., "low risk", "high risk").
    """
    # Build the user prompt
    patient_message = f"""Please respond **in English**. Determine the risk level of their pregnancy and return the pregnancy risk  with no explanation in English only

Age: {row['Age']}
SystolicBP: {row['SystolicBP']}mmHg
DiastolicBP: {row['DiastolicBP']}mmHg
Blood Sugar: {row['BS']}mmol/L
Body Temperature: {row['BodyTemp']}°F
Heart Rate: {row['HeartRate']} bpm\n


What is the pregnancy risk level for this individual?"""

    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": patient_message}]
        }
    ]

    # Tokenize with chat template
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        tokenize=True,
        return_dict=True,
    ).to("cpu") # Changed from "cuda" to "cpu"

    # Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=1.0,
        top_p=0.95,
        top_k=64,
    )

    # Decode and return
    return tokenizer.batch_decode(outputs)
    #, skip_special_tokens=True)[0].strip()

In [ ]:
# imports and libraries
# you can access this in the google drive as well
import pandas as pd
orginal_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI4Good/maternal+health+risk/uciirvone_maternal_health_data.csv')
data = orginal_data.copy()
data.loc[data.HeartRate == 7, "HeartRate"] = 70
data

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk
...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,high risk
1010,55,120,90,18.0,98.0,60,high risk
1011,35,85,60,19.0,98.0,86,high risk
1012,43,120,90,18.0,98.0,70,high risk


In [ ]:
from time import time
start = time()
row = data.iloc[0]
predict_pregnancy_risk(row, model, tokenizer)
end = time()
print(f"The total time is {(end - start)/60}")

The total time is 4.192168402671814


In [ ]:
sampler.chat("Age: 22, SystolicBP: 120, DiastolicBP: 90, BS: 7.0, BodyTemp: 98.0, HeartRate: 82. Predict the Risk Level.")

'1'

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import numpy as np

"""# Load the trained model
with open("model.pkl", "rb") as f:
    model = pickle.load(f)"""

app = FastAPI()

# Define the request body format
class InputData(BaseModel):
    age: int
    SystolicBP: float
    DiastolicBP: float
    BS: float
    BodyTemp: float
    heart_rate: int

@app.get("/")
def read_root():
    return {"message": "Welcome to the Pregnancy Risk Prediction API"}

@app.post("/predict")
def predict(data: InputData):
    # Convert input to the model's expected format
    input_array = """Age: {}
    SystolicBP: {}
    DiastolicBP: {}
    BS: {}
    BodyTemp: {}
    HeartRate: {}
    Predict the Risk Level.""".format(data.age, data.SystolicBP, data.DiastolicBP, data.BS, data.BodyTemp, data.heart_rate)
    # Make prediction
    prediction = sampler.chat(input_array)

    return {"prediction": prediction[0]}


In [ ]:
!pip install beam-client && beam configure default --token HcyYNc2DlPG6g3aH9Qpiv2tnxrycR93yDDOMkQVGaPc1gIpmuhRmpd387qzXdMtaxXU0rKjg_XIP009izEx25w==

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 45.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found 

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/AI4Good/gemmacomp

/content/drive/MyDrive/Colab Notebooks/AI4Good/gemmacomp


In [ ]:
[-!beam deploy "/content/drive/MyDrive/Colab Notebooks/AI4Good/gemmacomp/app.py":predict

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /usr/local/bin/beam:8 in <module>                                            │
│                                                                              │
│   5 from beam.cli.main import cli                                            │
│   6 if __name__ == '__main__':                                               │
│   7 │   sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                                      │
│   9                                                                          │
│                                                                              │
│ /usr/local/lib/python3.11/dist-packages/beam/cli/main.py:49 in cli           │
│                                                                              │
│   46 │   utils.check_version()                                               │
│   47 │                    

In [ ]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [11190] using WatchFiles
ERROR:    Error loading ASGI app. Could not import module "main".
^C


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/AI4Good

/content/drive/MyDrive/Colab Notebooks/AI4Good


In [ ]:
!pip install fastapi nest_asyncio pyngrok uvicorn


In [ ]:
"""from pyngrok import ngrok
import threading
import uvicorn
# Open a tunnel on port 8000


# Start FastAPI app
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()
public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url)"""

'from pyngrok import ngrok\nimport threading\nimport uvicorn\n# Open a tunnel on port 8000\n\n\n# Start FastAPI app\ndef run():\n    uvicorn.run(app, host="0.0.0.0", port=8000)\n\nthreading.Thread(target=run).start()\npublic_url = ngrok.connect(8000)\nprint("🚀 Public URL:", public_url)'

In [ ]:
!chmod +x start.sh


In [ ]:
from pyngrok import ngrok
import os
from google.colab import userdata

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
# You can store your authtoken securely in Colab's Secrets Manager and access it using userdata.get('NGROK_AUTH_TOKEN')
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# launch_ngrok_server.py
import nest_asyncio
from pyngrok import ngrok
from uvicorn import Config, Server
import asyncio
#from main import app  # This will import the FastAPI app

nest_asyncio.apply()

# Expose port 8001
public_url = ngrok.connect(8001)
print(f"🚀 Open your API docs here: {public_url}/docs")

# Run server
config = Config(app=app, host="0.0.0.0", port=8001, log_level="info")
server = Server(config=config)
asyncio.run(server.serve())


ERROR:pyngrok.process.ngrok:t=2025-07-30T07:11:32+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-30T07:11:32+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
pip freeze > requirements.txt

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
import os
from google.colab import userdata

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
# You can store your authtoken securely in Colab's Secrets Manager and access it using userdata.get('NGROK_AUTH_TOKEN')
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)